In [1]:
from typing import List, Tuple, Union
from itertools import zip_longest
import logging

from rdkit import Chem
import torch
import numpy as np

from chemprop.rdkit import make_mol

class Featurization_parameters:
    """
    A class holding molecule featurization parameters as attributes.
    """
    def __init__(self) -> None:

        # Atom feature sizes
        self.MAX_ATOMIC_NUM = 100
        self.ATOM_FEATURES = {
            'atomic_num': list(range(self.MAX_ATOMIC_NUM)),
            'degree': [0, 1, 2, 3, 4, 5],
            'formal_charge': [-1, -2, 1, 2, 0],
            'chiral_tag': [0, 1, 2, 3],
            'num_Hs': [0, 1, 2, 3, 4],
            'hybridization': [
                Chem.rdchem.HybridizationType.SP,
                Chem.rdchem.HybridizationType.SP2,
                Chem.rdchem.HybridizationType.SP3,
                Chem.rdchem.HybridizationType.SP3D,
                Chem.rdchem.HybridizationType.SP3D2
            ],
        }

        # Distance feature sizes
        self.PATH_DISTANCE_BINS = list(range(10))
        self.THREE_D_DISTANCE_MAX = 20
        self.THREE_D_DISTANCE_STEP = 1
        self.THREE_D_DISTANCE_BINS = list(range(0, self.THREE_D_DISTANCE_MAX + 1, self.THREE_D_DISTANCE_STEP))

        # len(choices) + 1 to include room for uncommon values; + 2 at end for IsAromatic and mass
        self.ATOM_FDIM = sum(len(choices) + 1 for choices in self.ATOM_FEATURES.values()) + 2
        self.EXTRA_ATOM_FDIM = 0
        self.BOND_FDIM = 14
        self.EXTRA_BOND_FDIM = 0
        self.REACTION_MODE = None
        self.EXPLICIT_H = False
        self.REACTION = False
        self.ADDING_H = False
        self.KEEP_ATOM_MAP = False

# Create a global parameter object for reference throughout this module
PARAMS = Featurization_parameters()


def reset_featurization_parameters(logger: logging.Logger = None) -> None:
    """
    Function resets feature parameter values to defaults by replacing the parameters instance.
    """
    if logger is not None:
        debug = logger.debug
    else:
        debug = print
    debug('Setting molecule featurization parameters to default.')
    global PARAMS
    PARAMS = Featurization_parameters()


def get_atom_fdim(overwrite_default_atom: bool = False, is_reaction: bool = False) -> int:
    """
    Gets the dimensionality of the atom feature vector.

    :param overwrite_default_atom: Whether to overwrite the default atom descriptors.
    :param is_reaction: Whether to add :code:`EXTRA_ATOM_FDIM` for reaction input when :code:`REACTION_MODE` is not None.
    :return: The dimensionality of the atom feature vector.
    """
    if PARAMS.REACTION_MODE:
        return (not overwrite_default_atom) * PARAMS.ATOM_FDIM + is_reaction * PARAMS.EXTRA_ATOM_FDIM
    else:
        return (not overwrite_default_atom) * PARAMS.ATOM_FDIM + PARAMS.EXTRA_ATOM_FDIM


def set_explicit_h(explicit_h: bool) -> None:
    """
    Sets whether RDKit molecules will be constructed with explicit Hs.

    :param explicit_h: Boolean whether to keep explicit Hs from input.
    """
    PARAMS.EXPLICIT_H = explicit_h

def set_adding_hs(adding_hs: bool) -> None:
    """
    Sets whether RDKit molecules will be constructed with adding the Hs to them.

    :param adding_hs: Boolean whether to add Hs to the molecule.
    """
    PARAMS.ADDING_H = adding_hs

def set_keeping_atom_map(keeping_atom_map: bool) -> None:
    """
    Sets whether RDKit molecules keep the original atom mapping.

    :param keeping_atom_map: Boolean whether to keep the original atom mapping.
    """
    PARAMS.KEEP_ATOM_MAP = keeping_atom_map

def set_reaction(reaction: bool, mode: str) -> None:
    """
    Sets whether to use a reaction or molecule as input and adapts feature dimensions.
 
    :param reaction: Boolean whether to except reactions as input.
    :param mode: Reaction mode to construct atom and bond feature vectors.

    """
    PARAMS.REACTION = reaction
    if reaction:
        PARAMS.EXTRA_ATOM_FDIM = PARAMS.ATOM_FDIM - PARAMS.MAX_ATOMIC_NUM - 1
        PARAMS.EXTRA_BOND_FDIM = PARAMS.BOND_FDIM
        PARAMS.REACTION_MODE = mode
        
def is_explicit_h(is_mol: bool = True) -> bool:
    r"""Returns whether to retain explicit Hs (for reactions only)"""
    if not is_mol:
        return PARAMS.EXPLICIT_H
    return False


def is_adding_hs(is_mol: bool = True) -> bool:
    r"""Returns whether to add explicit Hs to the mol (not for reactions)"""
    if is_mol:
        return PARAMS.ADDING_H
    return False


def is_keeping_atom_map(is_mol: bool = True) -> bool:
    r"""Returns whether to keep the original atom mapping (not for reactions)"""
    if is_mol:
        return PARAMS.KEEP_ATOM_MAP
    return True


def is_reaction(is_mol: bool = True) -> bool:
    r"""Returns whether to use reactions as input"""
    if is_mol:
        return False
    if PARAMS.REACTION: #(and not is_mol, checked above)
        return True
    return False


def reaction_mode() -> str:
    r"""Returns the reaction mode"""
    return PARAMS.REACTION_MODE


def set_extra_atom_fdim(extra):
    """Change the dimensionality of the atom feature vector."""
    PARAMS.EXTRA_ATOM_FDIM = extra


def get_bond_fdim(atom_messages: bool = False,
                  overwrite_default_bond: bool = False,
                  overwrite_default_atom: bool = False,
                  is_reaction: bool = False) -> int:
    """
    Gets the dimensionality of the bond feature vector.

    :param atom_messages: Whether atom messages are being used. If atom messages are used,
                          then the bond feature vector only contains bond features.
                          Otherwise it contains both atom and bond features.
    :param overwrite_default_bond: Whether to overwrite the default bond descriptors.
    :param overwrite_default_atom: Whether to overwrite the default atom descriptors.
    :param is_reaction: Whether to add :code:`EXTRA_BOND_FDIM` for reaction input when :code:`REACTION_MODE:` is not None
    :return: The dimensionality of the bond feature vector.
    """

    if PARAMS.REACTION_MODE:
        return (not overwrite_default_bond) * PARAMS.BOND_FDIM + is_reaction * PARAMS.EXTRA_BOND_FDIM + \
            (not atom_messages) * get_atom_fdim(overwrite_default_atom=overwrite_default_atom, is_reaction=is_reaction)
    else:
        return (not overwrite_default_bond) * PARAMS.BOND_FDIM + PARAMS.EXTRA_BOND_FDIM + \
            (not atom_messages) * get_atom_fdim(overwrite_default_atom=overwrite_default_atom, is_reaction=is_reaction)


def set_extra_bond_fdim(extra):
    """Change the dimensionality of the bond feature vector."""
    PARAMS.EXTRA_BOND_FDIM = extra


def onek_encoding_unk(value: int, choices: List[int]) -> List[int]:
    """
    Creates a one-hot encoding with an extra category for uncommon values.

    :param value: The value for which the encoding should be one.
    :param choices: A list of possible values.
    :return: A one-hot encoding of the :code:`value` in a list of length :code:`len(choices) + 1`.
             If :code:`value` is not in :code:`choices`, then the final element in the encoding is 1.
    """
    encoding = [0] * (len(choices) + 1)
    index = choices.index(value) if value in choices else -1
    encoding[index] = 1

    return encoding


def atom_features(atom: Chem.rdchem.Atom, functional_groups: List[int] = None) -> List[Union[bool, int, float]]:
    """
    Builds a feature vector for an atom.

    :param atom: An RDKit atom.
    :param functional_groups: A k-hot vector indicating the functional groups the atom belongs to.
    :return: A list containing the atom features.
    """
    if atom is None:
        features = [0] * PARAMS.ATOM_FDIM
    else:
        features = onek_encoding_unk(atom.GetAtomicNum() - 1, PARAMS.ATOM_FEATURES['atomic_num']) + \
            onek_encoding_unk(atom.GetTotalDegree(), PARAMS.ATOM_FEATURES['degree']) + \
            onek_encoding_unk(atom.GetFormalCharge(), PARAMS.ATOM_FEATURES['formal_charge']) + \
            onek_encoding_unk(int(atom.GetChiralTag()), PARAMS.ATOM_FEATURES['chiral_tag']) + \
            onek_encoding_unk(int(atom.GetTotalNumHs()), PARAMS.ATOM_FEATURES['num_Hs']) + \
            onek_encoding_unk(int(atom.GetHybridization()), PARAMS.ATOM_FEATURES['hybridization']) + \
            [1 if atom.GetIsAromatic() else 0] + \
            [atom.GetMass() * 0.01]  # scaled to about the same range as other features
        if functional_groups is not None:
            features += functional_groups
    return features

def atom_features_new(atom: Chem.rdchem.Atom, keep_features: List[bool] = [True]*8, functional_groups: List[int] = None) -> List[Union[bool, int, float]]:
    """
    Builds a feature vector for an atom.

    :param atom: An RDKit atom.
    :param keep_features: A boolean vector indicating which features to keep.
    :param functional_groups: A k-hot vector indicating the functional groups the atom belongs to.
    :return: A list containing the atom features.
    """
    if atom is None:
        return [0] * PARAMS.ATOM_FDIM

    # Define features and their corresponding keep flags
    feature_constructors = [
        (lambda atom: onek_encoding_unk(atom.GetAtomicNum() - 1, PARAMS.ATOM_FEATURES['atomic_num']), 0),
        (lambda atom: onek_encoding_unk(atom.GetTotalDegree(), PARAMS.ATOM_FEATURES['degree']), 1),
        (lambda atom: onek_encoding_unk(atom.GetFormalCharge(), PARAMS.ATOM_FEATURES['formal_charge']), 2),
        (lambda atom: onek_encoding_unk(int(atom.GetChiralTag()), PARAMS.ATOM_FEATURES['chiral_tag']), 3),
        (lambda atom: onek_encoding_unk(int(atom.GetTotalNumHs()), PARAMS.ATOM_FEATURES['num_Hs']), 4),
        (lambda atom: onek_encoding_unk(int(atom.GetHybridization()), PARAMS.ATOM_FEATURES['hybridization']), 5),
        (lambda atom: [1 if atom.GetIsAromatic() else 0], 6),
        (lambda atom: [atom.GetMass() * 0.01], 7)
    ]

    # Build features based on keep_features
    features = []
    for constructor, flag_index in feature_constructors:
        feature = constructor(atom)
        if not keep_features[flag_index]:
            feature = [0] * len(feature)
        features.extend(feature)

    if functional_groups is not None:
        features.extend(functional_groups)

    return features

def bond_features(bond: Chem.rdchem.Bond) -> List[Union[bool, int, float]]:
    """
    Builds a feature vector for a bond.

    :param bond: An RDKit bond.
    :return: A list containing the bond features.
    """
    if bond is None:
        fbond = [1] + [0] * (PARAMS.BOND_FDIM - 1)
    else:
        bt = bond.GetBondType()
        fbond = [
            0,  # bond is not None
            bt == Chem.rdchem.BondType.SINGLE,
            bt == Chem.rdchem.BondType.DOUBLE,
            bt == Chem.rdchem.BondType.TRIPLE,
            bt == Chem.rdchem.BondType.AROMATIC,
            (bond.GetIsConjugated() if bt is not None else 0),
            (bond.IsInRing() if bt is not None else 0)
        ]
        fbond += onek_encoding_unk(int(bond.GetStereo()), list(range(6)))
    return fbond

def bond_features_new(bond: Chem.rdchem.Bond, keep_features: List[bool] = [True]*4) -> List[Union[bool, int, float]]:
    """
    Builds a feature vector for a bond.

    :param bond: An RDKit bond.
    :param keep_features: A boolean vector indicating which features to keep.
    :return: A list containing the bond features.
    """
    if bond is None:
        fbond = [1] + [0] * (PARAMS.BOND_FDIM - 1)
    else:
        bt = bond.GetBondType()
        fbond = [0] # bond is not None
        bond_features_list = [
            (lambda bond: [bt == Chem.rdchem.BondType.SINGLE], 0),
            (lambda bond: [bt == Chem.rdchem.BondType.DOUBLE], 0),
            (lambda bond: [bt == Chem.rdchem.BondType.TRIPLE], 0),
            (lambda bond: [bt == Chem.rdchem.BondType.AROMATIC], 0),
            (lambda bond: [(bond.GetIsConjugated() if bt is not None else 0)], 1),
            (lambda bond: [(bond.IsInRing() if bt is not None else 0)], 2),
            (lambda bond: onek_encoding_unk(int(bond.GetStereo()), list(range(6))), 3)
        ]
        for constructor, flag_index in bond_features_list:
            feature = constructor(bond)
            if not keep_features[flag_index]:
                feature = [0] * len(feature)
            fbond.extend(feature)

    return fbond

/home/oscarwu/mambaforge/envs/chemprop/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from pprint import pprint

In [ ]:
# test bond feature fuction

In [98]:
smiles = 'CCO'
mol = Chem.MolFromSmiles(smiles)
bond = mol.GetBondWithIdx(0)

In [100]:
bond_feat_1 = bond_features(bond)

In [101]:
bond_feat_1

[0, True, False, False, False, False, False, 1, 0, 0, 0, 0, 0, 0]

In [103]:
bond_feat_2 = bond_features_new(bond)

In [104]:
bond_feat_2

[0, True, False, False, False, False, False, 1, 0, 0, 0, 0, 0, 0]

In [105]:
bond_feat_1 == bond_feat_2

True

In [ ]:
# test atom feature function

In [2]:
smiles = 'CCO'
mol = Chem.MolFromSmiles(smiles)
atom = mol.GetAtomWithIdx(0)

In [87]:
new_features = atom_features_new(atom)

In [88]:
len(new_features)

133

In [89]:
new_features == features

True

In [96]:
new_features_reduced = atom_features_new(atom, keep_features=[1]*8)

In [97]:
new_features_reduced

[0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0.12011]

In [3]:
features = atom_features(atom)

In [7]:
len(features)

133

In [6]:
pprint(features)

[0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0.12011]


In [9]:
import pandas as pd

In [12]:
atom_features= pd.read_csv("atom_types_hc.csv").values[:, 1:]

In [13]:
int(atom_features.shape[0]-1)

9

In [15]:
[9] * 20

[9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9]

In [21]:
bond_list = []
for i in range(5):
    for k in range(3):
        if i == k:
            bond_list.append(k)

In [22]:
bond_list

[0, 1, 2]

In [20]:
np.array(bond_list).reshape(1, -1)

array([[0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2]])

In [83]:
import numpy as np
from shap import links, PermutationExplainer
from copy import deepcopy

# Simple binary masker function
def binary_masker(binary_mask, x):
    masked_x = deepcopy(x)
    masked_x[binary_mask == 0] = 0
    return np.array([masked_x])

# Dummy model for illustration purposes
class DummyModel:
    def __call__(self, data):
        # Simple model that sums the input data and times the fisrt element by 2
        # return np.sum(data, axis=1, keepdims=True) + data[:, 0:1] * 2
        return data[:, 0:1] * 5 + data[:, 1:2] * 3

# Create the dummy model
model = DummyModel()

# Create the PermutationExplainer
explainer = PermutationExplainer(model=model, masker=binary_masker)

# Generate an example input to explain
example_input = np.array([[2, -5, 0, 1, 1], [1, -1, 0, 3, 4]])

# Explain the example input
# explanation = explainer(np.array([example_input]), max_evals=200)
explanation = explainer(example_input, max_evals=200)


print("SHAP values:", explanation.values)


SHAP values: [[ 10. -15.   0.   0.   0.]
 [  5.  -3.   0.   0.   0.]]


In [112]:
DummyModel()(np.array([[2, -5, 0, 1, 1], [1, -1, 0, 3, 4]]))

array([[-5],
       [ 2]])

In [81]:
np.array([example_input])

array([[[ 2, -5,  0,  1,  1],
        [ 1, -1,  0,  3,  4]]])

In [63]:
np.array([example_input])

array([[ 2, -5,  0,  1,  1]])

In [69]:
example_input

array([[ 2, -5,  0,  1,  1],
       [ 1, -1,  0,  3,  4]])

In [72]:
example_input.shape

(2, 5)

In [74]:
np.array([2, -5, 0, 1, 1]).shape

(5,)

In [76]:
np.array([np.array([2, -5, 0, 1, 1])]).shape

(1, 5)

In [106]:
# test mask out extra features
def mask_atom_features_extra(atom_features_extra: np.ndarray, keep_features: List[bool]) -> np.ndarray:
    """
    Masks certain features in atom_features_extra based on the keep_features boolean vector.

    :param atom_features_extra: A 2D numpy array with shape (num_atoms, feature_length) where feature_length is a multiple of 50.
    :param keep_features: A boolean vector indicating which features to keep. Each feature has a length of 50.
    :return: A masked 2D numpy array with the same shape as atom_features_extra.
    """
    num_atoms, feature_length = atom_features_extra.shape
    num_features = feature_length // 50  # Each feature has length 50

    if len(keep_features) != num_features:
        raise ValueError("Length of keep_features does not match the number of features in atom_features_extra.")

    # Create a mask to apply to atom_features_extra
    mask = np.ones_like(atom_features_extra)

    for i, keep in enumerate(keep_features):
        if not keep:
            mask[:, i*50:(i+1)*50] = 0

    # Apply the mask to atom_features_extra
    masked_features = atom_features_extra * mask

    return masked_features

In [118]:
# Dummy TrainArgs class for illustration
import sys
sys.path.insert(0, '/home/oscarwu/code/chemprop_developing')
from chemprop.args import TrainArgs

# Example SHAP Analysis with MPN

# Define the TrainArgs
args = TrainArgs()

# Create the MPN model
from chemprop.models.mpn import MPN
mpn_model = MPN(args)

# Example molecule (SMILES string)
smiles_list = ['CCO']

# # Define a batch (list of lists of molecules)
# batch = [[Chem.MolFromSmiles(smiles)] for smiles in smiles_list]

# Define dummy features batch, atom descriptors batch, and bond descriptors batch
features_batch = [np.random.rand(1, 100) for _ in range(len(smiles_list))]  # Example feature length of 100
atom_descriptors_batch = [np.random.rand(1, 100) for _ in range(len(smiles_list))]
bond_descriptors_batch = [np.random.rand(1, 50) for _ in range(len(smiles_list))]

# Define keep_features batches
extra_keep_features_batch = [[True] * 100 for _ in range(len(smiles_list))]  # Keep all features
extra_atom_keep_descriptors_batch = [[True] * 100 for _ in range(len(smiles_list))]
extra_bond_keep_descriptors_batch = [[True] * 50 for _ in range(len(smiles_list))]

# Create a SHAP explainer
def binary_masker(binary_mask, x):
    masked_x = deepcopy(x)
    masked_x[binary_mask == 0] = 0
    return np.array([masked_x])

# Define a wrapper for the model to match SHAP's expected input format
class MPNWrapper:
    def __init__(self, model):
        self.model = model

    def __call__(self, data):
        # Convert data to the expected input format for the MPN model
        batch = [[Chem.MolFromSmiles(smiles) for smiles in data]]
        features_batch = [np.random.rand(1, 100) for _ in data]
        output = self.model(batch, features_batch=features_batch, shap=True,
                            extra_keep_features_batch=extra_keep_features_batch,
                            extra_atom_keep_descriptors_batch=extra_atom_keep_descriptors_batch,
                            extra_bond_keep_descriptors_batch=extra_bond_keep_descriptors_batch)
        return output.detach().cpu().numpy()

# Create the MPN wrapper
model_wrapper = MPNWrapper(mpn_model)

# Create the PermutationExplainer
explainer = PermutationExplainer(model=model_wrapper, masker=binary_masker)

# Generate example input to explain
example_input = np.array(smiles_list)

# Explain the example input
explanation = explainer(example_input, max_evals=200)

print("SHAP values:", explanation.values)


TypeError: 'numpy.float64' object cannot be interpreted as an integer

In [117]:
MPNWrapper(mpn_model)(smiles_list)

TypeError: MPN.forward() got an unexpected keyword argument 'shap'